![](https://i.imgur.com/eBRPvWB.png)

# Generowanie poezji za pomocą RNN i PyTorch sylabami

[W tutorialu na rozgrzewkę](https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb) użyliśmy RNN, aby sklasyfikować nazwiska znak po znaku. Tym razem wygenerujemy tekst sylaba po sylabie.
```
Litwo! Ojczyzno moja! ty jesteś klucz wyziemu, 
To opugo cząciły tak lasu czeleta. 
Choć nie będzie mowę świeci się za tém, 
A Dozgon++ na Litwę przerzucił w okolicy, 
Dosyć się opicie przyciągnąć w pałacu; 

Tamdzini nawet mimo osobnych ogórki. 
Choć zwyciętunia, mimo pukle wyślą, 
Odemknął, wbiegł wyszedł, pewnie miłośnik łowił. 
Bo przekorza, i skrobiąc nabój do Warszawy. 
Dość co oddało plecie tak fawował, 
A tam się cukier wytaczać na nich wybująca. 

```

Ok, możesz zadać sobie pytanie, czy ten tutorial jest rzeczywiście praktyczny? Czemu nie? Modele generatywne tego typu stanowią fundament tłumaczenia maszynowego, opisywania obrazów, generowania odpowiedzi na pytania i wielu innych zastowań.

Zobacz [Sequence to Sequence Translation tutorial](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb) żeby nauczyć się więcej w tym temacie.

## Polecana lektura

Zakładam, że jest już zainstalowany PyTorch, znasz Python'a, oraz znasz pojęcie Tensor'ów:

* http://pytorch.org/ - instalacja PyTorch
* [Deep Learning with PyTorch: A 60-minute Blitz](http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) - Podstawy PyTorch
* [jcjohnson's PyTorch examples](https://github.com/jcjohnson/pytorch-examples) przykłady wykorzystania PyTorch
* [Introduction to PyTorch for former Torchies](https://github.com/pytorch/tutorials/blob/master/Introduction%20to%20PyTorch%20for%20former%20Torchies.ipynb) jeżeli znasz Lua Torch

Trochę wiedzy o RNN:

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) przykłady z życia wzięte
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) RNN i LSTM w pigułce

Zobacz także podobne tutoriale z serii:

* [Classifying Names with a Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/char-rnn-classification/char-rnn-classification.ipynb) używa RNN do klasyfikacji
* [Generating Names with a Conditional Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/conditional-char-rnn/conditional-char-rnn.ipynb) opierając się na tym modelu, dodaje kategorię jako dane wejściowe

## Załadowanie bibliotek

In [0]:
from pathlib import Path
import platform
import string
import random
import re
from IPython.core.display import display, HTML
import os
import psutil
import pickle
import warnings
import torch
import torch.nn as nn
from torch.autograd import Variable
import time, math
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

import matplotlib as mpl
mpl.style.use('default')
mpl.style.use('bmh')


## Preprocessing korpusu

In [0]:
dataset_path = Path('data/rnn_generator'); dataset_path
tmp_path = dataset_path / 'tmp/'
!mkdir -p $tmp_path

In [3]:
ls -lah $dataset_path/

total 12K
drwxr-xr-x 3 root root 4.0K Oct  8 14:39 ./
drwxr-xr-x 3 root root 4.0K Oct  8 14:39 ../
drwxr-xr-x 2 root root 4.0K Oct  8 14:39 tmp/


In [0]:
fn_corpus_char = dataset_path/'pan_tadeusz.txt'
fn_corpus_caps = dataset_path/'pan_tadeusz.caps1.txt'
fn_corpus_syl = dataset_path/'pan_tadeusz.syl1.txt'

Plik wejściowy (korpus) to duży plik tekstowy. 

In [5]:
!head -n 21 $fn_corpus_char

head: cannot open 'data/rnn_generator/pan_tadeusz.txt' for reading: No such file or directory


### Tokenizacja wielkich liter

Zamieniamy duże litery na małe dodając tokeny `_up_` (dla wyrazów pisanych wielkimi literami) lub `_cap_` (dla wyrazów pisanych z wielkiej litery).

In [0]:
def do_caps(ss):
  TOK_UP,TOK_CAP = ' _up_ ', ' _cap_ '
  res = []
  re_word = re.compile('\w')
  for s in re.findall(r'\w+|\W+', ss):
      res += ([TOK_UP,s.lower()] if (s.isupper() and (len(s)>2))
              else [TOK_CAP,s.lower()] if s.istitle()
              else [s.lower()])
  return ''.join(res)

In [7]:
corpus_tmp = fn_corpus_char.open('r').read()
corpus_tmp = do_caps(corpus_tmp)
fn_corpus_caps.open('w').write(corpus_tmp)

FileNotFoundError: ignored

In [0]:
!head -n 21 $fn_corpus_caps

### Podział korpusu na sylaby

Dzielimy korpus na sylaby programem `stemmer`.

In [0]:
platform_suffixes = {'Linux': 'linux', 'Darwin': 'macos'}
platform_suffix = platform_suffixes[platform.system()]
stemmer_bin = f'LD_PRELOAD="" bin/stemmer.{platform_suffix}'

In [0]:
!$stemmer_bin --help


In [0]:
!$stemmer_bin -s 7683 -v -d bin/stemmer2.dic -i $fn_corpus_caps -o $fn_corpus_syl

In [0]:
!head -n 21 $fn_corpus_syl

### Załadowanie do pamięci i tokenizacja

Ładujemy korpus do pamięci i tokenizujemy. Tworzymy też listę wszystkich tokenów `all_tokens`. Mamy już specjalne tokeny `_cap_` i `_up_`, zamieniamy znaki końca lini na token `_eol_` i dodajemy token `_unk_` na wypadek, gdybyśmy użyli sylaby (tokena), który nie wystąpił wcześniej w korpusie.

In [0]:
file = open(fn_corpus_syl).read()
file_len = len(file)
print('file_len =', file_len)

In [0]:
# taken from fastai/text.py

# remove +,- chars from punctuation set to keep syllables e.g.'--PO++' intact
# remove _ char to keep tokens intact
punctuation=re.sub('[_\+-]', '', string.punctuation)
re_tok = re.compile(f'([{punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')

def tokenize(s, repl_unk=True): 
  strings = re_tok.sub(r' \1 ', s).replace('\n', ' _eol_ ').split()
  if repl_unk:
    strings = [str2tok(s) for s in strings]
  return strings

file_tok = tokenize(file, repl_unk=False); len(file_tok), file_tok[:8]
file_tok_len = len(file_tok)

spec_tokens = ['_unk_', '_eol_', '_cap_', '_up_']

all_tokens = []
all_tokens.extend(spec_tokens)
all_tokens.extend(sorted(list(set(file_tok))))
n_tokens = len(all_tokens); print(n_tokens, all_tokens[:50])

tok2idx_dict = {tok: idx for (idx, tok) in enumerate(all_tokens)}

def str2tok(str) -> int:
  return str if tok2idx_dict.get(str, 0) else all_tokens[0]

def tok2idx(tok) -> int:
  return tok2idx_dict.get(tok, 0)

Przyda nam się funkcja do zakodowania dowolnego tekstu na listę zsylabizowanych tokenów:

In [0]:
def str2syl2tok(text):  
  fn_tmp_text_caps = Path(tmp_path / 'tmp_text_caps1.txt')
  fn_tmp_text_syl = Path(tmp_path / 'tmp_text_syl1.txt')
  
  text = do_caps(text)
  fn_tmp_text_caps.open('w').write(text)
  
  !$stemmer_bin -s 7683 -d bin/stemmer2.dic -i $fn_tmp_text_caps -o $fn_tmp_text_syl
  
  text_syl = fn_tmp_text_syl.open('r').read()
  
  # kill last \n eol char possibly added by stemmer
  if text_syl[-1] == '\n':
    text_syl = text_syl[:-1]

  text_tok = tokenize(text_syl, repl_unk=True)
    
  return text_tok

In [0]:
tekst = 'LITWO! Ojczyzno moja!\nTy jesteś jak zdrowie.\nIle cię trzeba cenić ble ble '
tekst_tok = str2syl2tok(tekst); print(tekst_tok)

Funkcje pomocnicze do zdekodowania listy tokenów na tekst:

In [0]:
def syl2str(a_list, delim='/'): 
  s = ' '.join(a_list)
  
  repl_list = [
      ('++ --', delim), 
  ]
  for repl in repl_list:
    s = s.replace(repl[0], repl[1])
  
  return s

print(syl2str(tekst_tok))

In [0]:
def decode_tokens(e_str):
  # decode _eol_, _cap_ and _up_
  # leave _unk_ token alone
  e_syl = e_str.split(' ')
  e_syl2 = []

  cap = False; up = False

  for syl in e_syl:
    if syl == '_eol_': syl = '\n'

    if syl not in ['_cap_', '_up_']:
      if cap == True: syl = syl.title(); cap = False
      if up == True: syl = syl.upper(); up = False        
      e_syl2.append(syl)

    if syl == '_cap_': cap = True
    if syl == '_up_': up = True

  return ' '.join(e_syl2)

print(decode_tokens(syl2str(tekst_tok, delim=''))[:300])

In [0]:
def fix_punctuation(s): 
  repl_list = [
      ('\n ', '\n'), 
      (' ,', ','),
      (' .', '.'),
      (' !', '!'),
      (' ?', '?'),
      (' ;', ';'),
      ('( ', '('),
      (' )', ')'),
      (' «', '«'),
      ('» ', '»'),
      (' :', ':')
  ]
  
  for repl in repl_list:
    s = s.replace(repl[0], repl[1])
  
  return s

print(fix_punctuation(decode_tokens(syl2str(tekst_tok, delim='')))[:300])

Sformatujmy zdekodowany tekst w HTML i zaznaczmy na czerwono sylaby, z których nie dało się skleić słów.

In [0]:
class X(str):
    def rpl(self, p, c='lightgray'):
        return X(self.replace(p, f'<font color="{c}">{p}</font>'))
    def rpl2(self, p, p2):
        return X(self.replace(p, p2))
      
def format_html(e_str):
  return X(e_str).rpl('/').rpl('--', c='red').rpl('++', c='red').rpl2('\n', '\n<br/>')

e_str = fix_punctuation(decode_tokens(syl2str(tekst_tok, delim='')))[:400]
e_html = format_html(e_str); display(HTML(e_html))

## Przygotowanie treningu

### GPU?

In [0]:
USE_GPU = torch.cuda.is_available(); 
# USE_GPU = False; 

print(f'USE_GPU={USE_GPU}')

def to_gpu(x, *args, **kwargs):
    return x.cuda(*args, **kwargs) if USE_GPU else x

### Budowa sieci rekurencyjnej

Ten model przyjmie jako wejściie token dla kroku $ t _ {- 1} $ i ma wyprowadzić następny token $ t $. Istnieją trzy warstwy - jedna warstwa liniowa, która koduje znak wejściowy do stanu wewnętrznego, jedna warstwa GRU (która może sama mieć wiele warstw), która działa na tym stanie wewnętrznym i stanie ukrytym, oraz warstwa dekodera, która wyprowadza rozkład prawdopodobieństwa.

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(to_gpu(torch.zeros(self.n_layers, 1, self.hidden_size)))

### Tensory wejściowe i docelowe

Aby stworzyć 'wejścia' z tego dużego ciągu danych, podzielimy go na kawałki po 400 sylab:

In [0]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_tok_len - chunk_len -1)
    end_index = start_index + chunk_len + 1
    return file_tok[start_index:end_index]
  
n_samples = file_tok_len // chunk_len; n_samples, file_tok_len

Każdy 'kawałek' zostanie przekształcony w tensor, a dokładnie w `LongTensor` (używany do wartości całkowitych), poprzez przepuszczenie wszystkich tokenów ciągu i wyszukiwanie indeksu każdej sylaby w `all_tokens`.

In [0]:
# Turn token list into list of longs
def tok_tensor(token_list):
    tensor = torch.zeros(len(token_list)).long()
    for c in range(len(token_list)):
        tensor[c] = tok2idx(token_list[c])
    
    return Variable(to_gpu(tensor))

In [0]:
tekst = 'Litwo! Ojczyzno moja! ty jesteś jak zdrowie;'
tekst_tok = str2syl2tok(tekst)
print(tekst_tok)
print(tok_tensor(tekst_tok))

Wreszcie możemy zmontować parę tensorów wejściowych i docelowych do treningu, z losowego kawałka. Wejściem zostaną wszystkie tokeny * aż do przedostatniego*, a celem (targetem) będą wszystkie tokeny * od drugiego*. Jeśli więc nasz kawałek to "abc", wejście będzie odpowiadać "ab", podczas gdy cel to "bc".

In [0]:
def random_training_set():  
    chunk = random_chunk()
    inp = tok_tensor(chunk[:-1])
    target = tok_tensor(chunk[1:])
    return inp, target
  
inp, target = random_training_set(); inp[:9], target[:9]

### Ewaluacja wyników

Aby ocenić sieć, będziemy podawać po jednym tokenie na raz, wykorzystywać wyjścia sieci jako rozkład prawdopodobieństwa dla następnego znaku i powtarzać. Aby rozpocząć generowanie, przekazujemy ciąg wstępny, aby rozpocząć budowanie stanu ukrytego, z którego następnie generujemy po jednym tokenie na raz.

In [0]:
def evaluate(prime_tokl=[all_tokens[1]], predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = tok_tensor(prime_tokl)
    predicted = list(prime_tokl)  # need a copy of the list

    # Use priming token list to "build up" hidden state
    for p in range(len(prime_tokl) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        
        # in pytorch 0.4.0 max, min fail if there are Infs or nans
        # https://github.com/pytorch/pytorch/issues/6996
        # in all pytorch versions multinomial fails if there are Infs or nans
        # https://github.com/pytorch/pytorch/issues/871
        # temp fix, kill Infs and nans
        # https://discuss.pytorch.org/t/how-to-set-inf-in-tensor-variable-to-0/10235
        output_dist[output_dist == float("Inf")] = 0
        output_dist[output_dist == float("nan")] = 0
        
        top_i = torch.multinomial(output_dist, 1)[0].item()
        
        # Add predicted token to the list and use as next input
        predicted_token = all_tokens[top_i]
        predicted.append(predicted_token)
        inp = tok_tensor([predicted_token])

    return predicted

## Trening sieci

Funkcja pomocnicza do wydrukowania upływającego czasu:

In [0]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

Główna funkcja treningowa:

In [0]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c].expand(1))

    loss.backward()
    decoder_optimizer.step()

    return loss.item() / chunk_len

Opcjonalny monitoring postępu treningu:

In [0]:
USE_VISDOM = False

vis = None
if USE_VISDOM:
    import visdom
    vis = visdom.Visdom(port=8890)

def vis_update_line_chart(vis, name, x, y, first_step):
    if not USE_VISDOM: return
    vis.line(Y=np.array([y]), X=np.array([x]), win=name, opts=dict(title=name),
             update=None if first_step else 'append')

def vis_update_text_win(vis, name, text):
    if not USE_VISDOM: return
    vis.text(text, win=name, opts=dict(title=name), append=False)

Wskaźnik liczby sylab, z których nie dało się skleić słów:

In [0]:
def bad_words(e_syl): e_str = syl2str(e_syl); return (e_str.count('++') + e_str.count('--')) / len(e_syl)

Następnie definiujemy parametry treningowe i rozpoczynamy trening:

In [0]:
n_epochs = 15
n_iters = n_epochs * n_samples
print_every = n_samples // 2
plot_every = n_samples // 4
hidden_size = 500
n_layers = 3
lr = 0.001

decoder = RNN(n_tokens, hidden_size, n_tokens, n_layers)
if USE_GPU:
    decoder.cuda()
print(decoder, flush=True)

decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
if USE_GPU:
    criterion.cuda()

all_losses = []
loss_avg = 0
all_bw = []
bw_avg = 0

iterable = range(1, n_iters + 1)
tqdm_ = tqdm(iterable, '', leave=False, dynamic_ncols=True, mininterval=1.0, ascii=True, miniters=1)
first_step = True

prime_tok = str2syl2tok('Litwo! Ojczyzno moja!')

In [0]:
start = time.time()

for it in tqdm_:
    epoch = 1 + it // n_samples
    loss = train(*random_training_set())       
    loss_avg += loss

    # current loss chart
    vis_update_line_chart(vis, 'loss', it, loss, it == 1)

    # bad words    
    bw = bad_words(evaluate(prime_tok, 100))
    bw_avg += bw

    # current bad words chart
    vis_update_line_chart(vis, 'bad_words', it, bw, it == 1)
    
    # progress_bar
    tqdm_.set_postfix({'epoch': f'{epoch}/{n_epochs}', 'loss': loss, 'bw': bw})
    text = f'&nbsp;<font color="red">{tqdm_}</font>'
    vis_update_text_win(vis, 'progress_bar', text)

    if it % print_every == 0:
        e_syl = evaluate(prime_tok, 1000)
        e_bw = bad_words(e_syl)
        stats_str = '\n[%s (%d %d %d%%) loss=%.4f bw=%.4f]' % (time_since(start), epoch, it, it / n_iters * 100, loss, e_bw)
        print(stats_str)
        
        e_str = fix_punctuation(decode_tokens(syl2str(e_syl, delim='')))
        e_html = format_html(e_str); display(HTML(e_html))
        print(flush=True)        
        
        text = f'<b>{stats_str}</b><br />{e_html}'
        vis_update_text_win(vis, 'evaluation', text)
        
        e_syl_path = tmp_path / 'e_syl.txt'
        e_syl_path.open('w').write(' '.join(e_syl))

    if it % plot_every == 0:
        vis_update_line_chart(vis, 'loss_avg', it, loss_avg / plot_every, first_step)
        vis_update_line_chart(vis, 'bad_words_avg', it, bw_avg / plot_every, first_step)
        all_bw.append(bw)
        bw_avg = 0
        first_step = False
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

### Kreślenie wartości straty

Wykreślanie historii straty z `all_losses` pokazuje uczenie sieci:

In [0]:
plt.figure()
plt.plot(all_losses)

### Zapis i odczyt sieci

#### Zapisanie sieci

In [0]:
# n_epochs=15

ALLTOKS, MODEL = ['all_tokens', 'model']
fn_pan_tadeusz = {ALLTOKS: f'all_tokens.n{n_tokens}.pan_tadeusz.p', 
                  MODEL: f'pan_tadeusz.h{hidden_size}.l{n_layers}.e{n_epochs}.gpu.torch'}
fn_dict = fn_pan_tadeusz; fn_dict

In [0]:
# save all_tokens
all_tokens_path = tmp_path / fn_dict[ALLTOKS]
pickle.dump(all_tokens, open(all_tokens_path, 'wb'))

warnings.filterwarnings('ignore')

# save model
model_path = tmp_path / fn_dict[MODEL]
torch.save(decoder, model_path)

In [0]:
ls -lah $tmp_path

In [0]:
decoder.state_dict

#### Załadowanie sieci

In [0]:
ls -lah $tmp_path

In [0]:
n_epochs=20
ALLTOKS, MODEL = ['all_tokens', 'model']
fn_pan_tadeusz = {ALLTOKS: f'all_tokens.n{n_tokens}.pan_tadeusz.p', 
                  MODEL: f'pan_tadeusz.h{hidden_size}.l{n_layers}.e{n_epochs}.gpu.torch'}
fn_dict = fn_pan_tadeusz; fn_dict

In [0]:
if True:
  all_tokens_path = tmp_path / fn_dict[ALLTOKS]
  print(f'all_tokens_path = {all_tokens_path}')
  all_tokens = pickle.load(open(all_tokens_path, 'rb'))
  n_characters = len(all_tokens)
  tok2idx_dict = {tok: idx for (idx, tok) in enumerate(all_tokens)}

  model_path = tmp_path / fn_dict[MODEL]
  decoder = torch.load(model_path)
  decoder.gru.flatten_parameters()
  print(f'model_path = {model_path}')
  print(decoder.state_dict)

## Ewaluacja w różnych "temperaturach"

W powyższej funkcji `evaluate`, za każdym razem, gdy dokonywana jest prognoza, wyjścia są dzielone przez przekazany argument "temperature". Użycie większej liczby sprawia, że wszystkie akcje są bardziej jednakowo prawdopodobne, a tym samym dają nam "bardziej losowe" wyniki. Użycie mniejszej wartości (mniejszej niż 1) sprawia, że wysokie prawdopodobieństwa przyczyniają się bardziej. Gdy ustawiamy temperaturę na zero, wybieramy tylko najbardziej prawdopodobne wyjścia.

Możemy zobaczyć te efekty poprzez dostosowanie argumentu `temperature`.


In [0]:
def print_eval(e_syl):
  display(HTML(format_html(fix_punctuation(decode_tokens(syl2str(e_syl, delim=''))))))

In [0]:
prime_tok = str2syl2tok('Litwo! Ojczyzno moja!')

In [0]:
print_eval(evaluate(prime_tok, 200, temperature=0.8))

Niższe temperatury daja mniejszą różnorodność, wybierając tylko bardziej prawdopodobne wyjścia:

In [0]:
print_eval(evaluate(prime_tok, 200, temperature=0.2))

Wyższe temperatury są bardziej różnorodne, wybierając mniej prawdopodobne wyjścia:

In [0]:
print_eval(evaluate(prime_tok, 200, temperature=1.4))

## Ćwiczenia

* Trenuj z własnym zestawem danych, np.
     * Tekst od innego autora
     * Posty na blogu
     * Kody źródłowe
* Zwiększ liczbę warstw i rozmiar sieci, aby uzyskać lepsze wyniki

**Następnie**: [Generating Names with a Conditional Character-Level RNN](https://github.com/spro/practical-pytorch/blob/master/conditional-char-rnn/conditional-char-rnn.ipynb)

## (debug) Monitorowanie maszyny wirtualnej

In [0]:
def print_memsize():
  process = psutil.Process(os.getpid())
  print(f'{process.memory_info().rss / 1024**3:.5} GB')

In [0]:
print_memsize()

In [0]:
!uptime